In [57]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

In [58]:
filename = "Case_PRB_Europe_Total_Deaths.csv"
df = pd.read_csv(filename)

look_back = 5
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    286 non-null    int64 
 1   date          286 non-null    object
 2   total_deaths  286 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.8+ KB
None


In [59]:
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
df.set_axis(df["date"], inplace = True)

In [60]:
death_data = df["total_deaths"].values
death_data = death_data.reshape((-1, 1))

split_percent = 0.80
split = int(split_percent*len(death_data))

death_train = death_data[:split]
death_test = death_data[split:]

date_train = df["date"][:split]
date_test = df["date"][split:]

# print(death_data)
print(len(death_train))
print(len(death_test))

228
58


In [61]:
train_generator = TimeseriesGenerator(death_train, death_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(death_test, death_test, length=look_back, batch_size=1)

In [62]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(look_back,1)))
model.add(Dropout(rate=0.2))
model.add(Dense(1))
model.compile(optimizer = "adam", loss="mse", metrics = ["accuracy"])

model.fit_generator(train_generator, epochs=30, verbose=1)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_29900\1823632347.py:7: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/30
12/12 [==============================] - 1s 4ms/step - loss: 13676615680.0000 - accuracy: 0.1839 
Epoch 2/30
12/12 [==============================] - 0s 3ms/step - loss: 10714643456.0000 - accuracy: 0.1839
Epoch 3/30
12/12 [==============================] - 0s 3ms/step - loss: 8831574016.0000 - accuracy: 0.1839 
Epoch 4/30
12/12 [==============================] - 0s 4ms/step - loss: 6811705856.0000 - accuracy: 0.1839
Epoch 5/30
12/12 [==============================] - 0s 3ms/step - loss: 1790338432.0000 - accuracy: 0.1839
Epoch 6/30
12/12 [==============================] - 0s 3ms/step - loss: 1436476032.0000 - accuracy: 0.1839
Epoch 7/30
12/12 [==============================] - 0s 3ms/step - loss: 973258624.0000 - accuracy: 0.1839
Epoch 8/30
12/12 [==============================] - 0s 3ms/step - loss: 629354944.0000 - accuracy: 0.1839
Epoch 9/30
12/12 [==============================] - 0s 3ms/step - loss: 1280819840.0000 - accuracy: 0.1839
Epoch 10/30
12/12 [================

In [63]:
prediction = model.predict(test_generator)

death_train = death_train.reshape((-1))
death_test = death_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = death_train,
    mode = "lines",
    name = "Data"
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = "lines",
    name = "Prediction"
)
trace3 = go.Scatter(
    x = date_test,
    y = death_test,
    mode="lines",
    name = "Ground Truth"
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()